In [6]:
import pandas as pd
import matplotlib.pyplot as plt
import plotly.express as px

import sys
sys.path.append("../")
import src.Resources as src

In [7]:
movies = pd.read_csv('../Data/Movies_clean.csv', index_col= 0)  
books = pd.read_csv('../Data/Books_clean.csv', index_col= 0)  

We drop any duplicates titles

In [8]:
movies["Stars"].fillna("Nostars",inplace=True)
movies.drop_duplicates(inplace=True)
movies = movies.drop_duplicates(subset="Title")
movies = movies.reset_index()
movies = movies.drop("index",axis=1)

books["Description"].fillna("No description",inplace=True)
books.drop_duplicates(inplace=True)
books = books.drop_duplicates(subset="Title")
books = books.reset_index()
books = books.drop("index",axis=1)

We split these columns by this character |

In [9]:
director = movies["Directors"].apply(lambda x: x.split("|"))
m_genre = movies["Genre"].apply(lambda x: x.split("|"))
stars = movies["Stars"].apply(lambda x: x.split("|"))
b_genre = books["Genre"].apply(lambda x: x.split("|"))

We explode some columns and get their unique values

In [10]:
n_directors = len(director.explode().unique())
movie_genres = len(m_genre.explode().unique())
movie_stars = len(stars.explode().unique())
book_genre = len(b_genre.explode().unique())
n_authors = books["Author"].nunique()
n_titles = books["Title"].nunique()
n_titles_m = movies["Title"].nunique()

These will be our big numbers

In [11]:
Bignumbers = pd.DataFrame([n_authors], columns=["Book_Authors"])
Bignumbers["Movie_directors"] = n_directors
Bignumbers["Movie_stars"] = movie_stars
Bignumbers["Book_titles"] = n_titles
Bignumbers["Movie_titles"] = n_titles_m
Bignumbers["Book_genres"] = book_genre
Bignumbers["Movie_genres"] = movie_genres

In [12]:
Bignumbers

,Book_Authors,Movie_directors,Movie_stars,Book_titles,Movie_titles,Book_genres,Movie_genres
0,5208,41338,129173,12483,77900,22,28


In [13]:
Bignumbers.to_csv("../Data/Bignumbers.csv")

We get some ratings for our graphs, i decided to filter by 1000 votes, feel free to filter higher or lower

In [14]:
movies_1 = movies[movies["Votes"]>1000]
books_1 = books[books["Votes"]>1000]

In [15]:
max_m =movies_1["Rating"].max()
min_m = movies_1["Rating"].min()
median_m = movies_1["Rating"].median()
mean_m = movies_1["Rating"].mean()

In [16]:
mean_m

6.3038062159839585

In [17]:
max_b = books_1["Rating"].max()
min_b = books_1["Rating"].min()
median_b = books_1["Rating"].median()
mean_b = books_1["Rating"].mean()

In [18]:
mean_b

8.011008438466037

These will be our metrics for our streamlit graphs

In [19]:
metric = ["Max","Min","Median","Mean","Max","Min","Median","Mean"]
rating = [max_m,min_m,median_m,mean_m,max_b,min_b,median_b,mean_b]
type = ["Movie","Movie","Movie","Movie","Book","Book","Book","Book"]


In [20]:
Metrics_movies_books = pd.DataFrame({"Metric": metric, "Rating": rating, "Type": type})
Metrics_movies_books.to_csv("../Data/Metrics_movies_books.csv")

In [21]:
books['Genre'] = books['Genre'].apply(src.map_genre)
movies["Genre"] = movies["Genre"].apply(src.map_movie_genre)

In [22]:
books.dropna(inplace=True)
movies.dropna(inplace=True)
books["Genre"] = books["Genre"].apply(lambda x: x.split("|"))
movies["Genre"] = movies["Genre"].apply(lambda x : x.split("|"))

In [23]:
genre_books = books["Genre"].explode().value_counts().reset_index()
genre_movies = movies["Genre"].explode().value_counts().reset_index()

In [25]:
genre_books["Type"]= "Book"
genre_movies["Type"] = "Movie"

In [26]:
genre_total = pd.concat([genre_books,genre_movies])

Here i want to filter with votes so i only get the best books and movies, this will be used to do a filter that shows the 3 best movies and books of each genre

In [36]:
movies_2 = movies[movies["Votes"]>80000]
books_2 = books[books["Votes"]>80000]

In [30]:
books_exploded = pd.DataFrame({"Image":books_2["Image"],"Title": books_2["Title"],"Author":books_2["Author"], "Rating": books_2["Rating"],"Description":books_2["Description"],"Genre":books_2["Genre"]})
movies_exploded = pd.DataFrame({"Image":movies_2["Image"],"Title": movies_2["Title"],"Directors":movies_2["Directors"], "Rating": movies_2["Rating"],"Description": movies_2["Description"],"Genre":movies_2["Genre"]})


In [31]:
books_exploded = books_exploded.explode("Genre").reset_index(drop=True)
movies_exploded = movies_exploded.explode("Genre").reset_index(drop=True)

In [32]:
books_exploded.to_csv("../Data/Books_exploded.csv")
movies_exploded.to_csv("../Data/Movies_exploded.csv")
genre_total.to_csv("../Data/genre_total.csv")